In [ ]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
from datetime import datetime, timedelta
import bcolz
import sys
sys.path.append('../pyx')
import cmodels as cmod
from tqdm import tqdm
import rasterio
import matplotlib.pyplot as plt
import h5py
import scipy.ndimage
import os
try:
    import cPickle as pickle
except:
    import pickle

In [ ]:
peq = pd.read_pickle('../data/amazon/peq.pkl')
df_x = pd.read_pickle('../data/amazon/df_x.pkl')
df_ws = pd.read_pickle('../data/amazon/amazon.pkl')
df_imerg = pd.read_pickle('../data/amazon/imerg.pkl')
df_pet = pd.read_pickle('../data/amazon/pet.pkl')
ca_gpm = bcolz.open(rootdir = '../data/amazon/ca_gpm', mode = 'r')

In [ ]:
lat0, lat1, lon0, lon1 = -21, 6, -80, -52 # bounding box in degrees
gpm_res = 0.1 # IMERG resolution in degrees
pet_res = 1 / 120 # PET resolution in degrees (0.008333)
acc_res = 1 / 120 # flow accumulation resolution in degrees (0.008333)

In [ ]:
def get_p_imerg(df_ws, df_imerg, ca_gpm, odr):
    first_time = True
    pix_sum = 0.
    for odr_i, this_odr in enumerate(df_ws.index):
        if this_odr in odr:
            this_mask = df_ws.loc[this_odr, 'mask']
            this_p = ca_gpm[odr_i] * np.sum(this_mask)
            pix_sum += np.sum(this_mask)
            if first_time:
                first_time = False
                p_imerg = this_p
            else:
                p_imerg = p_imerg + this_p
    p_imerg = Series(data = p_imerg / pix_sum, index = df_imerg.index)
    if p_imerg.index[0].to_datetime().minute != 0:
        p_imerg = p_imerg.iloc[1:]
    p_imerg = p_imerg.reindex(pd.date_range(p_imerg.index[0], p_imerg.index[-1], freq = '30min'))
    p_imerg = p_imerg.interpolate()
    p_imerg = pd.rolling_mean(p_imerg, window = 3, min_periods = 3, center = True)
    p_imerg = p_imerg.iloc[1::2].dropna()
    p_imerg.index = (p_imerg.index - timedelta(minutes = 30)).tz_localize('UTC')
    return p_imerg

def get_pet(df_ws, df_imerg, df_pet, odr):
    pix_sum = 0.
    months = np.array([this_date.month for this_date in df_imerg.index], dtype = 'uint8')
    this_pet = np.empty(len(df_imerg.index))
    pet = np.zeros_like(this_pet)
    for this_odr in df_ws.index:
        if this_odr in odr:
            this_mask = df_ws.loc[this_odr, 'mask']
            for i in range(1, 12 + 1):
                this_pet[np.where(months == i)] = df_pet.loc[i, this_odr]
            this_pet = this_pet * np.sum(this_mask)
            pix_sum += np.sum(this_mask)
            pet = pet + this_pet
    pet = Series(data = pet / pix_sum, index = df_imerg.index)
    if pet.index[0].to_datetime().minute != 0:
        pet = pet.iloc[1:]
    pet = pet.reindex(pd.date_range(pet.index[0], pet.index[-1], freq = '30min'))
    pet = pet.interpolate()
    pet = pd.rolling_mean(pet, window = 3, min_periods = 3, center = True)
    pet = pet.iloc[1::2].dropna()
    pet.index = (pet.index - timedelta(minutes = 30)).tz_localize('UTC')
    return pet

In [ ]:
try:
    df_qsim = pd.read_pickle('../data/amazon/df_qsim.pkl')
except:
    df_qsim = DataFrame()
    for odr in tqdm(df_ws.index):
        this_x = [df_x[df_x['odr'] == odr]['x' + str(x_i + 1)].values[0] for x_i in range(4)]
        this_peq = DataFrame()
        this_peq['e'] = get_pet(df_ws, df_imerg, df_pet, [odr])
        this_peq['p'] = get_p_imerg(df_ws, df_imerg, ca_gpm, [odr])
        q_mod = cmod.gr4h(this_x)
        q_sim = q_mod.run([this_peq.p.values, this_peq.e.values])
        q_sim = np.append(q_sim, q_mod.run([this_peq.p.values, this_peq.e.values]))
        q_sim = np.append(q_sim, q_mod.run([this_peq.p.values, this_peq.e.values]))
        q_sim = np.append(q_sim, q_mod.run([this_peq.p.values, this_peq.e.values]))
        q_sim = q_sim * df_x[df_x['odr'] == odr]['weight'].values[0]
        q_sim2 = Series(data = q_sim[-len(this_peq.index):], index = this_peq.index)
        if odr not in df_qsim.columns:
            df_qsim[odr] = q_sim2
        else:
            df_qsim.loc[:, odr] = df_qsim.loc[:, odr] + q_sim2
        this_odr = odr
        delay = 0.
        while len(this_odr.split(',')) > 1:
            next_odr = str(this_odr.split(',')[:-1])[1:-1].replace("'", '').replace(' ', '')
            delay += df_x[df_x['odr'] == this_odr]['x5+'].values[0] - df_x[df_x['odr'] == next_odr]['x5+'].values[0]
            q_sim2 = cmod.delay([delay]).run([q_sim])
            q_sim3 = Series(data = q_sim2[-len(this_peq.index):], index = this_peq.index)
            if next_odr not in df_qsim.columns:
                df_qsim[next_odr] = q_sim3
            else:
                df_qsim.loc[:, next_odr] = df_qsim.loc[:, next_odr] + q_sim3
            this_odr = next_odr
    df_qsim.to_pickle('../data/amazon/df_qsim.pkl')

In [ ]:
rc_df = DataFrame()
rc_df['h'] = peq['q']
rc_df['q'] = df_qsim['0'] * (5267179 * 1e6 * 1e-3) / 3600
rc_df = rc_df.dropna().sort(['h'])

plt.figure()
plt.scatter(rc_df['h'], rc_df['q'])
plt.plot(rc_df['h'], rc_df['q'])
plt.xlabel('Stage ' + r'($m$)')
plt.ylabel('Discharge ' + r'($m^{3}/s$)')
plt.title('Amazon River rating curve at (lat, lon) = 1.95S / 53.84W')
plt.savefig('../data/amazon/rating_curve.png', bbox_inches='tight')

In [ ]:
lat0, lat1, lon0, lon1 = -21, 6, -80, -52 # bounding box in degrees
with rasterio.drivers():
    with rasterio.open('../data/amazon/sa_dem_30s_hillshade.tif') as src:
        data, = src.read()
a, b = round((lat1 - src.affine[5]) / src.affine[4]), round((lat0 - src.affine[5]) / src.affine[4])
c, d = round((lon0 - src.affine[2]) / src.affine[0]), round((lon1 - src.affine[2]) / src.affine[0])
dem = data[a:b, c:d]
#dem = np.where(dem > 0, dem, np.nan)
dem = dem - np.min(dem)
dem = dem / np.max(dem)

In [ ]:
smin = np.log10(0.00001)
smax = np.log10(0.15)

vmin = 0
vmax = smax - smin

x0 = int((lon0 - (-180)) / gpm_res)
x1 = int((lon1 - (-180)) / gpm_res)
y0 = int((90 - lat1) / gpm_res)
y1 = int((90 - lat0) / gpm_res)

try:
    qsim_keep = pickle.load(open('../data/amazon/jpg/qsim_keep.pkl', 'rb'))
    diff = np.load('../data/amazon/jpg/diff.npy')
except:
    qsim_keep = {}
date_i = 0
daterange = pd.date_range(df_qsim.index[0], df_qsim.index[-1], freq = '30min', tz = 'UTC')
for this_date in tqdm(daterange):
    if not os.path.exists('../data/amazon/jpg/img' + str(date_i) + '.jpg'):
        # streamflow:
        if this_date.minute == 0:
            diff = np.zeros((int((lat1 - lat0) / acc_res), int((lon1 - lon0) / acc_res)))
            for this_head in df_qsim.columns:
                xx0, xx1, yy0, yy1 = df_ws.loc[this_head, 'acc_view']
                this_acc = np.array(df_ws.loc[this_head, 'acc_mask'], dtype = 'float')
                if date_i >= 2 and np.max(this_acc) > 0.:
                    qsim_diff = df_qsim.loc[this_date, this_head] - qsim_keep[this_head]
                    radius = 3
                    if qsim_diff < 0:
                        radius = radius
                    else:
                        radius = round(qsim_diff  * radius * 1000) + radius
                    circle = np.zeros((2 * radius, 2 * radius)).astype('uint8')
                    cx, cy = radius, radius
                    y, x = np.ogrid[-radius: radius, -radius: radius]
                    index = x**2 + y**2 <= radius**2
                    circle[cy - radius:cy + radius, cx - radius:cx + radius][index] = 1
                    streamflow = np.log10(this_acc * df_qsim.loc[this_date, this_head] / np.max(this_acc))
                    y, x = streamflow.shape
                    streamflow2 = np.empty((y + 2 * radius, x + 2 * radius))
                    streamflow2[:, :] = -np.inf
                    streamflow2[radius:-radius, radius:-radius] = streamflow
                    streamflow = scipy.ndimage.maximum_filter(streamflow2, footprint = circle)
                    streamflow = np.where(np.isinf(streamflow), smin, streamflow)
                    streamflow = np.where(streamflow < smin, smin, streamflow) - smin
                    diff[yy0 - radius:yy1 + radius, xx0 - radius:xx1 + radius] = np.maximum(diff[yy0 - radius:yy1 + radius, xx0 - radius:xx1 + radius], streamflow)
                qsim_keep[this_head] = df_qsim.loc[this_date, this_head]
        # rainfall:
        if os.path.exists('../data/amazon/shrink/p_' + str(this_date).replace(' ', '_')[:-6] + '.npy'):
            p = np.load('../data/amazon/shrink/p_' + str(this_date).replace(' ', '_')[:-6] + '.npy')
            p = np.log10(p)
            p = np.where(np.isinf(p), -3, p)
            p = (np.where(p < -3, -3, p) + 3) / 20
            rainfall = np.empty((int((lat1 - lat0) / acc_res), int((lon1 - lon0) / acc_res)))
            for y in range(rainfall.shape[0]):
                for x in range(rainfall.shape[1]):
                    rainfall[y, x] = p[int(y / 12), int(x / 12)]
        # add streamflow, precipitation and shaded hills to image:
        img = rainfall + dem / 5
        img = np.where(diff > img, diff, img)
        plt.close('all')
        fig = plt.figure(frameon=False)
        fig.set_size_inches(img.shape[1] / 500, img.shape[0] / 500)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        plt.text(0, 0, this_date, fontsize=15, color='white', verticalalignment='top', horizontalalignment='left')
        cax = ax.imshow(img, interpolation = 'nearest', vmin=vmin, vmax=vmax, cmap = 'jet', origin = 'upper', aspect='auto')
        cbaxes = fig.add_axes([0.01, 0.07, 0.5, 0.02]) 
        sm = plt.cm.ScalarMappable(cmap='jet', norm=plt.Normalize(vmin=vmin, vmax=vmax))
        sm._A = []
        m3ps = [100, 1000, 10000, 100000]
        ticks = [[np.log10(i * 3600 / (5267179 * 1e6 * 1e-3)) - smin] for i in m3ps]
        cbar = fig.colorbar(cax=cbaxes, orientation='horizontal', mappable = sm, ticks = ticks)
        cbar.ax.set_xticklabels([r'$10^{' + str(int(i)) + '}$' for i in np.log10(m3ps)])
        cbar.set_label(label=r'$m^{3}/s$', size=12, weight='bold', color = 'white')
        cbytick_obj = plt.getp(cbar.ax.axes, 'xticklabels')
        plt.setp(cbytick_obj, size = 10, color='white')
        plt.savefig('../data/amazon/jpg/img' + str(date_i) + '.jpg', dpi = 200)
        pickle.dump(qsim_keep, open('/home/david/data/gross/amazon/jpg/qsim_keep.pkl', 'wb'))
        np.save('../data/amazon/jpg/diff', diff)
    date_i += 1